In [6]:
#packages
import pandas as pd
import json
import requests
from random import randint
from lxml import html
import re
import time
import multiprocessing as mp
#from pathos.multiprocessing import ProcessingPool as Pool #python package which enables multiprocessing on child classes
import time
import demjson

#utilities
from util.http_utility import get_http_headers
from util.user_agent import get_random_ua
from  util.spider import Spider

# Food4Thought: Russia

url: https://www.russianfood.com/recipes/bytype/?fid=103#rcp_list

The website contains a lot of recipes from different countries, fid=103 is for Russia. 
Russianfood.com has IP blocking, need time sleep... and disable multithreading.

In [7]:
class RussianSpider(Spider): #inherit Spider Class

    def __init__(self, main_page, seeds, listing, attrs, available_json, header=None ):
        super().__init__(main_page,seeds,listing,attrs, available_json, header)

    def scrape_one_item(self,url):
        """To scrape one item from a given url

        Args:
            url (str): url of the item

        Returns:
            dict: dict of the item
        """
      
        session = requests.Session() 
        session = requests.Session() 
        #randomize user-agent in every request
        random_ua =  get_random_ua()
        header_template = self.header
        header_template['user-agent'] = random_ua
        session.headers.update(header_template)

        #add random time sleep for this website
        random_second = randint(2,4)
        time.sleep(random_second)

        #add home page url to the obtained item url if item url is not complete
        if self.main_page not in url:
            res = session.get(self.main_page + url)
        else:
            res = session.get(url)
        
        #default valuess
        name, total_time, ingredients, instructions, servings, category, prep_time, cook_time = '','','','','','','',''

        if res.status_code == 200:
            try:
                #print(res.status_code)
                doc = html.document_fromstring(res.text)
                #set default values for variables
                

                #Name
                if self.check_normalize_space(self.attrs['name']):
                    name = doc.xpath(self.attrs['name'])
                else: name = doc.xpath(self.attrs['name'])[0]

                #Total Time
                if self.check_normalize_space(self.attrs['total_time']):
                    total_time = doc.xpath(self.attrs['total_time'])
                else:  total_time = doc.xpath(self.attrs['total_time'])[0]

                #ingredients
                ingredients = doc.xpath(self.attrs['ingredients'])
                clean_ingredients = []
                for ing in ingredients:
                    ing = ing.replace('\t','')
                    ing = ing.replace('\r','')
                    ing = ing.replace('\n','')
                    clean_ingredients.append(ing)

                #print(clean_ingredients)
                #ingredients = ''.join(clean_ingredients)
                #ingredients = ingredients.strip()
                # ingredient_str = ''
                # for ing in ingredients:
                #     if  ' ' not in str(ing):
                #         ingredient_str += ing

                ingredients = list(filter(None, clean_ingredients))
                

                #instructions
                instructions = doc.xpath(self.attrs['instructions'])
                #print(instructions)
                #instructions= ''.join(instructions)
                
                # instructions_str = ''
                # for ins in instructions:
                #     if  ' ' not in str(ins): 
                #         instructions_str += ins
                # instructions = instructions_str

                # if self.check_normalize_space(self.attrs['instructions']):
                #     instructions = doc.xpath(self.attrs['instructions'])
                # else:  instructions = doc.xpath(self.attrs['instructions'])[0]
            
                #servings
                if self.attrs['servings']:
                    if self.check_normalize_space(self.attrs['servings']):
                        servings = doc.xpath(self.attrs['servings'])
                    else: servings = doc.xpath(self.attrs['servings'])[0]

                #category
                if self.attrs['category']:
                    if self.check_normalize_space(self.attrs['category']):
                        category = doc.xpath(self.attrs['category'])
                    else: category = doc.xpath(self.attrs['category'])[0]

                #prep time
                if self.attrs['prep_time']:
                    if self.check_normalize_space(self.attrs['prep_time']):
                        prep_time = doc.xpath(self.attrs['prep_time'])
                    else: prep_time = doc.xpath(self.attrs['prep_time'])[0]
                
                #cooking time
                if self.attrs['cook_time']:
                    if self.check_normalize_space(self.attrs['cook_time']):
                        cook_time = doc.xpath(self.attrs['cook_time'])
                    else: cook_time = doc.xpath(self.attrs['cook_time'])[0]
                
                return {'name': name, 'total_time': total_time, 'ingredients': ingredients, 'instructions': instructions, 'servings': servings,
                'category': category, 'prep_time': prep_time, 'cook_time': cook_time,}
            
            except Exception as e:
                print(e)
                print('something is wrong for item: {}'.format(url)) 
                raise Exception(e)
            #     return {'name': name, 'total_time': total_time, 'ingredients': ingredients, 'instructions': instructions, 'servings': servings,
            # 'category': category, 'prep_time': prep_time, 'cook_time': cook_time,}

        else:
            print('Cannot open one recipe url, status code = {}, url = {}'.format(res.status_code, url))
            raise Exception('Cannot open one menu url, status code = {}, url = {}'.format(res.status_code, url))
            # return {'name': name, 'total_time': total_time, 'ingredients': ingredients, 'instructions': instructions, 'servings': servings,
            # 'category': category, 'prep_time': prep_time, 'cook_time': cook_time,}
        
    def start_scrape(self, max_pages=100, multithread=True):
        """Get all recipe items from the seeds given. Can choose to enable multiprocessing.
        Multiprocessing is used to immediately scrape all menu items from a list of menu urls, but be careful of blockers.

         Args:
            max_pages (int): maximum number of pages to scrape, default =100 

        Returns:
            list: list containing the url of items for all seeds for all pages
        """

        COLUMN_NAMES = ['name','ingredients', 'total_time','instructions', 'servings','category','prep_time','cook_time']
        all_items = []
        try:
            for seed_url in self.seeds:
                if self.listing['next']['type'] == 'url':
                    for i in range(max_pages):
                        listing_items = []
                        page_url = seed_url + self.listing['next']['next_page_str'].format(i+1)
                        print('spider is scraping page: {}'.format(i+1))
                        #print('spider is scraping url: ', page_url)
                        listing_items = list(set(self.get_items_from_page(page_url))) #convert to set to remove duplicate urls
                        if not listing_items:
                            break #last page
                        if multithread: 
                            #pool = Pool() #initialize Pathos multiprocessing pool
                            pool = mp.Pool(mp.cpu_count()) #initialize multiprocessing pool
                            results  = pool.map(self.scrape_one_item, [url for url in listing_items])
                            print('scraping - multithreaded, n items: ', len(listing_items))
                            pool.close() #close the multiprocessing  pool
                        else:
                            results = []
                            for item_url in listing_items:
                                item = self.scrape_one_item(item_url)
                                results.append(item)
                                                
                        all_items += results

                        # for item_url in listing_items:
                        #     item = self.scrape_one_item(item_url)

                            #item_df = pd.DataFrame([item], columns=item.keys())
                            #df = pd.concat([df,item_df], ignore_index=True)
                # 
                if self.listing['next']['type'] == 'custom_modify_url':
                    for i in range(max_pages):
                        listing_items = []
                        page_url = seed_url.format(i+1)
                        print('spider is scraping page: {}'.format(i+1))
                        #print('spider is scraping url: ', page_url)
                        listing_items = list(set(self.get_items_from_page(page_url))) #convert to set to remove duplicate urls
                        if not listing_items:
                            break #last page
                        if multithread: 
                            #pool = Pool() #initialize Pathos multiprocessing pool
                            pool = mp.Pool(mp.cpu_count()) #initialize multiprocessing pool
                            results  = pool.map(self.scrape_one_item, [url for url in listing_items])
                            print('scraping - multithreaded, n items: ', len(listing_items))
                            pool.close() #close the multiprocessing  pool
                        else:
                            results = []
                            for item_url in listing_items:
                                item = self.scrape_one_item(item_url)
                                results.append(item)
                                                
                        all_items += results
       
            return all_items

        except Exception as e:
            print(e)
            return all_items
    
    def get_items_from_page(self, page_url):
        """Get all item urls from the current page listing

        Args:
            page_url (str): url of the page

        Returns:
           list: list containing the url of items in the current page
        """
        session = requests.Session() 
        #randomize user-agent in every request
        random_ua =  get_random_ua()
        header_template = self.header
        header_template['user-agent'] = random_ua
        session.headers.update(header_template)
        res = session.get(page_url)
        if res.status_code == 200:
            doc = html.document_fromstring(res.text)
            urls = doc.xpath(self.listing['items'])
            #print('got items from url: ', page_url)
            return urls
        else:
            #raise Exception('Cannot open the menu url, status code = {}'.format(res.status_code))
            print('Cannot open the menu url, status code = {}, url= {}'.format(res.status_code, page_url))
            return []


In [8]:
attrs = {
        'name':         'normalize-space(//title)',
        'ingredients':  '//tr[contains(@class,"ingr_tr_")]//text()',
        'total_time':   'normalize-space(//div[@class="sub_info"]/div[@class="el"][2])',
        'instructions': '//div[@class="step_n"]//p//text()',
        'servings':     'normalize-space(//div[@class="sub_info"]/div[@class="el"][1])',
        'category':     '',
        'prep_time':    '',
        'cook_time':    '',
}

listing={'items': '//div[contains(@class,"recipe_list_new")]//div[contains(@class,"title")]/a/@href', 'next': { 'next_page_str': '&page={}', 'type': 'url'}}
seeds = ['https://www.russianfood.com/recipes/bytype/?fid=103']
available_json={}

In [9]:
#setup variables
custom_header_template = { #setup custom header because romania requires certain headers
        'referer': 'https://www.google.com/',
        'Accept-Language': '*',
        'Accept-Encoding': '*',
        'Accept': '*',
        'user-agent': ''}
ru_spider= RussianSpider('https://www.russianfood.com/', seeds= seeds, listing =listing,attrs= attrs, available_json=available_json, header=custom_header_template,)
#print(romania_spider.attrs)
ru_spider.scrape_one_item('https://www.russianfood.com/recipes/recipe.php?rid=164402') #try out scraping one item

{'name': 'Рецепт: Картофельные драники с чесноком (без яиц и муки) на RussianFood.com',
 'total_time': '\xa030 мин (ваши 30 мин)',
 'ingredients': ['Картофель - 800 г',
  'Лук репчатый - 150 г',
  'Сметана - 1 ст. ложка',
  'Соль - 0,5 ч. ложки',
  'Перец молотый (смесь перцев) - 1 щепотка',
  'Масло растительное - 4 ст. ложки'],
 'instructions': ['Подготовьте все для драников - картофель, лук, чеснок, сметану, специи и растительное масло для жарки.',
  'Картофель очистите от кожуры. Репчатый лук и чеснок очистите от шелухи.',
  'Репчатый лук и чеснок натрите на мелкой терке в глубокую миску.',
  'Туда же натрите на мелкой тёрке картофель, периодически перемешивая его с луком. Добавьте соль, перемешайте.',
  'Картофельно-луковую массу откиньте на дуршлаг или мелкое сито, сливая жидкость в миску. Оставьте на 2-3 минуты, слегка придавливая массу лопаткой. Сильно выжимать не нужно, чтобы драники получились сочными.',
  'У меня получилось примерно 120-130 мл жидкости. Аккуратно слейте верх

In [10]:
result_list = ru_spider.start_scrape(max_pages=65, multithread=False) #max page is only 65, have to disable multithreading because of  blocking

spider is scraping page: 1
spider is scraping page: 2
spider is scraping page: 3
spider is scraping page: 4
spider is scraping page: 5
spider is scraping page: 6
spider is scraping page: 7
spider is scraping page: 8
Document is empty
something is wrong for item: /recipes/recipe.php?rid=82973
Document is empty


In [11]:
result_list

[{'name': 'Рецепт: Мясо по-французски на RussianFood.com',
  'total_time': '',
  'ingredients': ['Свинина (ошеек) - 1,2 кг',
   'Картофель - 1 кг',
   'Лук репчатый - 2 шт.',
   'Сыр российский - 250 г',
   'Майонез - 50 г',
   'Соль - по вкусу',
   'Перец - по вкусу',
   'Масло растительное - для смазывания формы'],
  'instructions': ['Как приготовить мясо по-французски:',
   '\r\n',
   '\r\nБерем свиную шею, нарезаем её стейками толщиной примерно 1 см. Раскладываем на рабочей поверхности и приправляем солью и перцем с обеих сторон.',
   'Картофель очищаем и нарезаем ломтиками толщиной примерно полсантиметра.',
   'Лук нарезаем кольцами.',
   'Сыр натираем на терке.',
   'Слегка смазываем форму для запекания растительным маслом и выкладываем в нее слоями подготовленные ингредиенты. Слои идут в следующей последовательности: мясо, картошка (не забудьте её подсолить), майонез, лук. ',
   '\r\nОтправляем мясо по-французски в заранее разогретую до 200 градусов духовку. Запекаем мясо с карт

In [12]:
result_df = pd.DataFrame(result_list)

In [13]:
result_df

,name,total_time,ingredients,instructions,servings,category,prep_time,cook_time
0,Рецепт: Мясо по-французски на RussianFood.com,,"[Свинина (ошеек) - 1,2 кг, Картофель - 1 кг, Л...","[Как приготовить мясо по-французски:, \r\n, \r...",,,,
1,Рецепт: Пирожки из кефирного теста на RussianF...,,"[Кефир - 200 мл, Яйца - 2 шт., Маргарин - 50 г...",[],1 час 30 мин,,,
2,Рецепт: Сельдь домашнего соления на RussianFoo...,,"[Сельдь — 2 шт., Вода — 1 литр, Соль — 4 ст. л...",[Перед засолкой сельди размораживаем ее и хоро...,8 порций,,,
3,Рецепт: Картофельные драники с чесноком (без я...,30 мин (ваши 30 мин),"[Картофель - 800 г, Лук репчатый - 150 г, Смет...","[Подготовьте все для драников - картофель, лук...",4 порции,,,
4,Рецепт: Картофельная запеканка с фаршем на Rus...,1 час 10 мин (ваши 30 мин),"[Картофель - 2 кг, Фарш мясной - 500 г, Молоко...",[Как приготовить картофельную запеканку с фарш...,6 порций,,,
...,...,...,...,...,...,...,...,...
345,Рецепт: Бабушкины блины на RussianFood.com,40 мин (ваши 40 мин),"[Яйцо куриное - 3 шт., Молоко (теплое) - 700 м...","[Ингредиенты перед Вами., В миску разбить яйца...",10 порций,,,
346,"Рецепт: Кальмары, фаршированные крабовыми пало...",1 час (ваши 30 мин),"[Кальмары - 2 шт., Крабовые палочки - 80 г, Мо...","[Подготовить все продукты по списку., \r\nВоду...",3 порции,,,
347,Рецепт: Маковые коржики на RussianFood.com,1 час 20 мин (ваши 25 мин),"[Мука - 2,5 стакана, Сметана - 3 ст. ложки, Ма...","[Перед тем как приготовить маковые коржики, по...",8 порций,,,
348,Рецепт: Рассольник с клецками на RussianFood.com,1 час 30 мин,"[Грудинка говяжья - 600 г, Картофель - 2-3 шт....","[Как приготовить рассольник с клецками:, \r\n,...",6 порций,,,


In [14]:
result_df.to_csv('data/russia/russian_food_com.csv')

I got blocked from accessing this website whilst I was scraping the 7th page. Changing IP addresses with VPN gives the same problem.